<a href="https://colab.research.google.com/github/Yed-hu/Projects/blob/main/Cargo_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CargoProject").getOrCreate()

In [2]:
from google.colab import files
uploaded = files.upload()

#upload the data files
# customers
# shipments
# transactions
# feedbacks

Saving transactions.csv to transactions.csv
Saving shipments.csv to shipments.csv
Saving delivery_feedback.csv to delivery_feedback.csv
Saving customers.csv to customers.csv


In [3]:
customers_df = spark.read.csv("/content/customers.csv", header=True, inferSchema= True)
shipments_df = spark.read.csv("/content/shipments.csv", header=True, inferSchema= True)
transactions_df = spark.read.csv("/content/transactions.csv", header= True, inferSchema=True)
feedbacks_df = spark.read.csv("/content/delivery_feedback.csv", header= True, inferSchema= True)

In [4]:
customers_df.printSchema()
customers_df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: long (nullable = true)
 |-- dob: date (nullable = true)
 |-- nationality: string (nullable = true)
 |-- region: string (nullable = true)
 |-- country_of_residence: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- account_creation_date: date (nullable = true)

+---+-----------+----------+---------+--------------------+------------+----------+-----------+------------------+--------------------+--------------------+-------------+---------------------+
|_c0|customer_id|first_name|last_name|               email|       phone|       dob|nationality|            region|country_of_residence|          occupation|customer_type|account_creation_date|
+---+-----------+----------+---------+--------------------+-----------

In [5]:
shipments_df.show(5)
shipments_df.printSchema()

+---+-----------+-----------+--------------+-------------------+-----------+----------------+-----------------+-------------------+-------------+-------------+---------------+---------------+-----------------+-------------------+-----------+------------+
|_c0|shipment_id|customer_id|origin_country|destination_country|origin_city|destination_city|           weight|             volume|shipment_date|delivery_date|delivery_status|  shipment_type|customs_clearance|cargo_provider_name| cargo_type|service_type|
+---+-----------+-----------+--------------+-------------------+-----------+----------------+-----------------+-------------------+-------------+-------------+---------------+---------------+-----------------+-------------------+-----------+------------+
|  0|   SHIP7575|     CUST35|         India|      United States|     Kollam|      South Jane|46.21837223975306|  4.888725669313287|   2024-04-20|   2024-04-27|        Delayed|       Document|             true|                DHL|Air Fr

In [6]:
transactions_df.printSchema()
transactions_df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- shipment_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- transaction_amount: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- transaction_status: string (nullable = true)

+---+--------------+-----------+-----------+----------------+------------------+--------------+------------------+
|_c0|transaction_id|customer_id|shipment_id|transaction_date|transaction_amount|payment_method|transaction_status|
+---+--------------+-----------+-----------+----------------+------------------+--------------+------------------+
|  0|      TRAN8113|      CUST9|   SHIP3991|      2025-01-20| 71494.66204052712|           COD|              Paid|
|  1|     TRAN14864|    CUST172|  SHIP10578|      2023-08-03| 1022.844756058191|   Credit Card|              Paid|
|  2|     TRAN13481|    CUST356|  SHIP11633|      2023-05-2

In [7]:
feedbacks_df.show(5)
feedbacks_df.printSchema()

+---+-----------+-----------+-----------+-------------+---------------+--------------+-------------+----------------+
|_c0|feedback_id|shipment_id|customer_id|delivery_time|delivery_rating|issue_reported|   issue_type|complaint_status|
+---+-----------+-----------+-----------+-------------+---------------+--------------+-------------+----------------+
|  0|     FD8969|   SHIP7770|    CUST747|            6|              3|          true|Wrong Address|         Pending|
|  1|    FD15977|  SHIP23248|    CUST539|            1|              1|         false|         NULL|            NULL|
|  2|      FD164|  SHIP11172|    CUST327|            3|              4|          true|         Lost|      Unresolved|
|  3|    FD24984|   SHIP3414|    CUST729|            3|              5|          true|       Damage|        Resolved|
|  4|    FD13944|  SHIP24348|    CUST668|            7|              1|          true|         Lost|         Pending|
+---+-----------+-----------+-----------+-------------+-

In [8]:
customers_df = customers_df.drop('_c0')
shipments_df = shipments_df.drop('_c0')
transactions_df = transactions_df.drop('_c0')
feedbacks_df = feedbacks_df.drop('_c0')

In [9]:
customers_df.show(5)

+-----------+----------+---------+--------------------+------------+----------+-----------+------------------+--------------------+--------------------+-------------+---------------------+
|customer_id|first_name|last_name|               email|       phone|       dob|nationality|            region|country_of_residence|          occupation|customer_type|account_creation_date|
+-----------+----------+---------+--------------------+------------+----------+-----------+------------------+--------------------+--------------------+-------------+---------------------+
|    CUST655|     Jason|  Simmons|jasonsimmons114@g...|917026855092|1992-07-17|     Indain|          Thrissur|       United States|Working Professional|      Regular|           2023-01-20|
|    CUST143|    Robert|  Stewart|robertstewart754@...|917110053353|1972-04-01|     Indain|    Pathanamthitta|             Belgium|             Student|          New|           2022-08-28|
|    CUST433|   Michael|   Turner|michaelturner32@g...|

In [10]:
from pyspark.sql.functions import to_date

customers_df = customers_df.withColumn("dob",to_date("dob")).withColumn("account_creation_date", to_date("account_creation_date"))

shipments_df = shipments_df.withColumn("shipment_date", to_date("shipment_date")) \
                           .withColumn("delivery_date", to_date("delivery_date"))

transactions_df = transactions_df.withColumn("transaction_date", to_date("transaction_date"))

In [11]:
customers_df.groupBy("nationality").count().orderBy("count", ascending = False).show()

+-----------+-----+
|nationality|count|
+-----------+-----+
|     Indain|  750|
+-----------+-----+



In [12]:
customers_df.groupBy("country_of_residence").count().orderBy("count", ascending = False).show()

+--------------------+-----+
|country_of_residence|count|
+--------------------+-----+
|              Greece|   34|
|              Sweden|   33|
|             Belgium|   33|
|             Germany|   31|
|              Kuwait|   31|
|              Norway|   31|
|              Canada|   31|
|             Denmark|   30|
|           Australia|   30|
|             Bahrain|   30|
|         Switzerland|   29|
|      United Kingdom|   29|
|           Singapore|   28|
|              France|   28|
|               Italy|   28|
|        South Africa|   28|
|            Malaysia|   27|
|       United States|   26|
|             Ireland|   26|
|                Oman|   26|
+--------------------+-----+
only showing top 20 rows



In [13]:
from pyspark.sql.functions import year, current_date

customers_df = customers_df.withColumn("age",2025-year("dob"))
customers_df.select("first_name","last_name","age").show(5)

+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|     Jason|  Simmons| 33|
|    Robert|  Stewart| 53|
|   Michael|   Turner| 31|
| Alexander|   Benson| 26|
|     Megan|     Tran| 31|
+----------+---------+---+
only showing top 5 rows



In [14]:
customers_df.groupBy("customer_type").count().show()

+-------------+-----+
|customer_type|count|
+-------------+-----+
|      Regular|  249|
|          New|  264|
|          VIP|  237|
+-------------+-----+



In [15]:
customers_df.groupBy("region").count().orderBy("count", ascending = False).show()

+------------------+-----+
|            region|count|
+------------------+-----+
|          Kottayam|   65|
|    Pathanamthitta|   63|
|            Kollam|   62|
|         Kozhikode|   61|
|            Kannur|   59|
|         Kasaragod|   58|
|            Idukki|   57|
|          Thrissur|   53|
|Thiruvananthapuram|   52|
|        Malappuram|   50|
|           Wayanad|   46|
|         Ernakulam|   45|
|          Palakkad|   44|
|         Alappuzha|   35|
+------------------+-----+



In [16]:
customers_df.groupBy("occupation").count().show()

+--------------------+-----+
|          occupation|count|
+--------------------+-----+
|             Student|  196|
|Working Professional|  190|
|            Business|  184|
|             Retired|  180|
+--------------------+-----+



In [17]:
shipments_df.groupBy("destination_country").count().orderBy("count",ascending=False).show()

+-------------------+-----+
|destination_country|count|
+-------------------+-----+
|            Belgium| 1160|
|             Greece| 1111|
|             Kuwait| 1109|
|             Sweden| 1076|
|             Canada| 1056|
|            Germany| 1038|
|             Norway| 1007|
|            Denmark|  988|
|     United Kingdom|  975|
|        Switzerland|  965|
|          Australia|  963|
|            Bahrain|  962|
|          Singapore|  961|
|             France|  950|
|       South Africa|  941|
|              Italy|  938|
|            Ireland|  892|
|           Malaysia|  872|
|       Saudi Arabia|  855|
|               Oman|  845|
+-------------------+-----+
only showing top 20 rows



In [25]:
from pyspark.sql.functions import datediff, avg, round

shipments_df = shipments_df.withColumn("delivery_time_days", datediff("delivery_date","shipment_date"))

shipments_df.select("shipment_id", "delivery_time_days").show(5)
shipments_df.select(round(avg("delivery_time_days"))).show()

+-----------+------------------+
|shipment_id|delivery_time_days|
+-----------+------------------+
|   SHIP7575|                 7|
|  SHIP11804|                 4|
|   SHIP5588|                 7|
|  SHIP13133|                 9|
|   SHIP4343|                 5|
+-----------+------------------+
only showing top 5 rows

+---------------------------------+
|round(avg(delivery_time_days), 0)|
+---------------------------------+
|                              6.0|
+---------------------------------+



In [33]:
from pyspark.sql.functions import sum

transactions_df.select(round(sum("transaction_amount")/1e7,2).alias("total_transaction_amountin_crores")).show()   # converting into crores (1e7)


+---------------------------------+
|total_transaction_amountin_crores|
+---------------------------------+
|                           125.97|
+---------------------------------+

